<a href="https://colab.research.google.com/github/KANISHRED/KANISHRED/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
print(os.listdir("archive"))

['rating.csv', 'tag.csv', 'genome_scores.csv', 'genome_tags.csv', 'link.csv', 'movie.csv']


In [ ]:
import pandas as pd

In [ ]:
# import movie data set and look at columns
movie = pd.read_csv("archive/movie.csv")
movie.columns

Index(['movieId', 'title', 'genres'], dtype='object')

In [ ]:
# what we need is that movie id and title
movie = movie.loc[:,["movieId","title"]]
movie.head(10)


,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


In [ ]:
# import rating data and look at columsn
rating = pd.read_csv("archive/rating.csv")
rating.columns


Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [ ]:
# what we need is that user id, movie id and rating
rating = rating.loc[:,["userId","movieId","rating"]]
rating.head(10)


,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5
5,1,112,3.5
6,1,151,4.0
7,1,223,4.0
8,1,253,4.0
9,1,260,4.0


In [ ]:
# then merge movie and rating data
data = pd.merge(movie,rating,on = "movieId")


In [ ]:
# now lets look at our data
data.head(10)


,movieId,title,userId,rating
0,1,Toy Story (1995),3,4.0
1,1,Toy Story (1995),6,5.0
2,1,Toy Story (1995),8,4.0
3,1,Toy Story (1995),10,4.0
4,1,Toy Story (1995),11,4.5
5,1,Toy Story (1995),12,4.0
6,1,Toy Story (1995),13,4.0
7,1,Toy Story (1995),14,4.5
8,1,Toy Story (1995),16,3.0
9,1,Toy Story (1995),19,5.0


In [ ]:
data.shape


(20000263, 4)

In [ ]:
data = data.iloc[:1000000,:]


In [ ]:
# lets make a pivot table in order to make rows are users and columns are movies. And values are rating
pivot_table = data.pivot_table(index = ["userId"],columns = ["title"],values = "rating")
pivot_table.head(10)


title,Ace Ventura: When Nature Calls (1995),Across the Sea of Time (1995),"Amazing Panda Adventure, The (1995)","American President, The (1995)",Angela (1995),Angels and Insects (1995),Anne Frank Remembered (1995),Antonia's Line (Antonia) (1995),Assassins (1995),Babe (1995),...,Unforgettable (1996),Up Close and Personal (1996),"Usual Suspects, The (1995)",Vampire in Brooklyn (1995),Waiting to Exhale (1995),When Night Is Falling (1995),"White Balloon, The (Badkonake sefid) (1995)",White Squall (1996),Wings of Courage (1995),"Young Poisoner's Handbook, The (1995)"
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
movie_watched = pivot_table["Bad Boys (1995)"]
similarity_with_other_movies = pivot_table.corrwith(movie_watched)  # find correlation between "Bad Boys (1995)" and other movies
similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False)
similarity_with_other_movies.head()

title
Bad Boys (1995)                        1.000000
Headless Body in Topless Bar (1995)    0.723747
Last Summer in the Hamptons (1995)     0.607554
Two Bits (1995)                        0.507008
Shadows (Cienie) (1988)                0.494186
dtype: float64

In [ ]:
### Rec Sys 2

In [ ]:
!pip install scikit-surprise

In [ ]:
# Import required libraries
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [ ]:
data.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),3,4.0
1,1,Toy Story (1995),6,5.0
2,1,Toy Story (1995),8,4.0
3,1,Toy Story (1995),10,4.0
4,1,Toy Story (1995),11,4.5


In [ ]:
# Load Reader library
reader = Reader()

# get just top 100K rows for faster run time
data_svd = Dataset.load_from_df(data[['userId', 'movieId', 'rating']][:100000], reader)

# Use the SVD algorithm.
svd = SVD()

# Compute the RMSE of the SVD algorithm
cross_validate(svd, data_svd, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9320  0.9277  0.9262  0.9287  0.0025  
MAE (testset)     0.7199  0.7196  0.7051  0.7149  0.0069  
Fit time          0.70    0.84    0.73    0.76    0.06    
Test time         0.25    0.26    0.16    0.22    0.05    


{'test_rmse': array([0.93202728, 0.92772153, 0.92622676]),
 'test_mae': array([0.71985551, 0.71962599, 0.70511723]),
 'fit_time': (0.701218843460083, 0.8377630710601807, 0.7280361652374268),
 'test_time': (0.2500641345977783, 0.26044797897338867, 0.16017580032348633)}

In [ ]:
data

,movieId,title,userId,rating
0,1,Toy Story (1995),3,4.0
1,1,Toy Story (1995),6,5.0
2,1,Toy Story (1995),8,4.0
3,1,Toy Story (1995),10,4.0
4,1,Toy Story (1995),11,4.5
...,...,...,...,...
999995,147,"Basketball Diaries, The (1995)",93300,4.0
999996,147,"Basketball Diaries, The (1995)",93365,3.0
999997,147,"Basketball Diaries, The (1995)",93411,2.5
999998,147,"Basketball Diaries, The (1995)",93476,3.0


## To find all the movies rated as 5 stars by user with userId = 93300

In [ ]:
dataset_rec = data[(data['userId'] == 93300) & (data['rating'] == 5)]
#dataset_rec = dataset_rec.set_index('movieId')
dataset_rec.head(10)

,movieId,title,userId,rating
115597,6,Heat (1995),93300,5.0
872197,110,Braveheart (1995),93300,5.0


In [ ]:
svd

In [ ]:
dataset_rec['movieId'].apply(lambda x: svd.predict(93300, x).est)

115597    3.883448
872197    3.531455
Name: movieId, dtype: float64

In [ ]:
dataset_rec_02 = data[(data['userId'] == 93300) & (data['rating'] == 4)]

In [ ]:
dataset_rec_02['movieId'].apply(lambda x: svd.predict(93300, x).est)

260588    3.531455
699159    3.531455
999995    3.531455
Name: movieId, dtype: float64

In [ ]:
df_all = pd.DataFrame(data['movieId'])['movieId'].apply(lambda x: svd.predict(93300, x).est)

In [ ]:
df_all.sort_values()

86976    2.841732
86410    2.841732
86411    2.841732
86412    2.841732
86413    2.841732
           ...   
33126    3.955120
33125    3.955120
33124    3.955120
33146    3.955120
0        3.955120
Name: movieId, Length: 1000000, dtype: float64